#### Imports

In [542]:
import PyPDF2
import csv
import re
import json
import os
import us
import numpy as np

#### Scrape key to facility codes
- Very inclusive info about keys, many things not keys included

In [3]:
year = 1998

pdfFileObj = open('{0}/directory_{0}.pdf'.format(year), 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
print("Number of pages:", pdfReader.numPages)

Number of pages: 568


In [582]:
abbr_keys = ["TX", "ML", "PV", "OS", "DT",\
             "OD", "OR", "IO", "ID", "IR", "RD", "RR",\
             "AD", "DD", "HV", "PW",\
             "SS", "GH", "PH", "OH", "TC", "HH", "CM", "CH", \
             "CO", "SG", "SC", "CJ",\
             "MC", "MD", "FG", "PI"]

#### List of States in the U.S.

In [476]:
states = "Alabama,\
Alaska,\
Arizona,\
Arkansas,\
California,\
Colorado,\
Connecticut,\
Delaware,\
Florida,\
Georgia,\
Hawaii,\
Idaho,\
Illinois,\
Indiana,\
Iowa,\
Kansas,\
Kentucky,\
Louisiana,\
Maine,\
Maryland,\
Massachusetts,\
Michigan,\
Minnesota,\
Mississippi,\
Missouri,\
Montana,\
Nebraska,\
Nevada,\
New Hampshire,\
New Jersey,\
New Mexico,\
New York,\
North Carolina,\
North Dakota,\
Ohio,\
Oklahoma,\
Oregon,\
Pennsylvania,\
Rhode Island,\
South Carolina,\
South Dakota,\
Tennessee,\
Texas,\
Utah,\
Vermont,\
Virginia,\
Washington,\
West Virginia,\
Wisconsin,\
Wyoming"

In [477]:
states = states.split(",")

In [478]:
states

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

#### Test on a page of data
- data starts on page 12 (getPage(11))
- split to each city, then split into data centers, clean city name and keys!

- Function to match city names

In [888]:
def fuzzy_search(search_key, text):
    #print("Searching {}...".format(search_key))
    #print("in", text)
    l = len(search_key)
    search_key = search_key.lower()
    strictness = int(l/4) + 1
    for pos in re.finditer(r"[\.,]", text):
        word_to_test = text[pos.start() - l: pos.start()].lower()
        i = 0
        diff = 0
        while i < l:
            if word_to_test[i] != search_key[i]:
                diff += 1
            i += 1
        if diff < strictness:
            return (pos.start() - l, pos.start())      

In [889]:
#test
text = "DemopolisWestAl.abamaMentalDcmopolis.AL36732"
fuzzy_search("DEMOPOLIS", text)

(27, 36)

- Fucntion to split words

In [890]:
def split_cap_words(string):
    string = re.sub(r"([A-z()])([A-Z0-9])([a-z0-9])", r"\1 \2\3", string)
    string = re.sub(r"([0-9])([A-z])", r"\1 \2", string)
    string = re.sub(r"([A-z])([0-9])", r"\1 \2", string)
    return  string#(\w)([A-Z])([a-z])

In [891]:
#test
split_cap_words("PGBHAPinalCountyHispanicCouncil712NorthMainStreet")

'PGBHA Pinal County Hispanic Council 712 North Main Street'

In [892]:
split_cap_words("CommunityCareNetwork(CCN)JewishFamilyandChildrensServices6376WestBellRoad")

'Community Care Network(CCN) Jewish Familyand Childrens Services 6376 West Bell Road'

In [893]:
split_cap_words("MohaveMentalHealthClinicOutpatientandDayTreatment2135Highway9SSuites125and241")

'Mohave Mental Health Clinic Outpatientand Day Treatment 2135 Highway 9 S Suites 125 and 241'

In [894]:
split_cap_words("564NorthIdahoRoadSuite9")

'564 North Idaho Road Suite 9'

- Split data into a list containing a list of facilities in that city

In [911]:
def split_data(rawdata, firstcity):
    '''
    Roughly split data into each facility, returns list of list.
    '''
    data = []
    ###city = firstcity
    while len(rawdata) > 50:
        #print("\n", rawdata)
        temp_city = []
        ###m = 1
        
        ###if rawdata[:3].isupper():
        #find the capitalized string
        m = re.search("[A-Z]+", rawdata).end()
        city = rawdata[:m - 1]
        #clean state name from the city name if present
        for state in states:
            if state.lower() in city.lower():
                city = city[len(state):]
                rawdata = rawdata[len(state):]
                m = m - len(state)
        #identify city name
        if fuzzy_search(city, rawdata[m - 1:]) == None:
            i = 2
            while (i < m) and (fuzzy_search(rawdata[:i], rawdata[i:]) == None):
                i += 1
            m = i + 1
            city = rawdata[:i]
                
        rawdata = rawdata[m - 1:]
        a, b = fuzzy_search(city, rawdata)
        state_clean = split_cap_words(rawdata[a:b])
        temp_city.append(state_clean)
        #while there is more facilities in this city (still can find city name in the string)
        while fuzzy_search(city, rawdata) != None:
            temp_facil = []
            a, b = fuzzy_search(city, rawdata)
            m = re.search("[A-Z]{2}[A-Z/][A-Z/]", rawdata[b:]).start() #start of the keys
            temp_facil.append(rawdata[:b+m])
            rawdata = rawdata[b+m:]
            temp_facil.append('')
            j = 0
            #append keys to the dataframe
            while (rawdata[j:j+2].replace("1", "I").replace("l", "I").replace("0", "O").replace("5", "S")\
                   .replace("i", "I") in abbr_keys) and (rawdata[j+2].replace("1", "I").replace("l", "I")\
            .replace("0", "O").replace("5", "S").replace("i", "I").isupper() or rawdata[j+2] == '/'):
                temp_facil[-1] += ' ' + \
                rawdata[j:j+2].replace("1", "I").replace("l", "I").replace("0", "O").replace("5", "S")
                if rawdata[j+2] == "/":
                    j += 3
                else:
                    j += 2
                    
            rawdata = rawdata[j:]
            temp_facil[-1] = temp_facil[-1].strip()
            temp_city.append(temp_facil)
        data.append(temp_city)
        i = 0
    
    print(data)
    while i < len(data):
        city = data[i]
        if not city[0][0].isupper():
            init = data[i - 1][-1][-1][-2:].capitalize()
            data[i - 1][-1][-1] = data[i - 1][-1][-1][:-3]
            data[i][0] = init + data[i][0]
        i += 1
    return data

In [912]:
def split_center_street(cn_sa):
    '''
    Split a string containing center name and street address into two.
    Inputs:
        cn_sa(str)
    Returns: a list of two terms
    '''
    
    m = re.finditer(r"\d+", cn_sa)
    try:
        probe = next(m)
        span = probe.span()
        if span[1] == len(cn_sa):
            cn_sa = split_cap_words(cn_sa).split()
            return[' '.join(cn_sa[:-2]), ' '.join(cn_sa[-2:])]
        elif span[0] != 0:
            return ([split_cap_words(cn_sa[:span[0]]), split_cap_words(cn_sa[span[0]:])])
    except StopIteration:
        if cn_sa[-4:] == "Road":
            cn_sa = split_cap_words(cn_sa).split()
            return[' '.join(cn_sa[:-2]), ' '.join(cn_sa[-2:])]
    return [split_cap_words(cn_sa), '']


def split_address(csp):
    '''
    Split a string containing city, state and postal code.
    Inputs:
        csp(str): a string containing city, state and postal code
    Returns: a list of three items
    '''
    PASS

In [902]:
cn_sa = "SMMHCIncMentalHealthCtr/SubstAbuseServices564NorthIdahoRoadSuite9"
split_center_street(cn_sa)

['SMMHC Inc Mental Health Ctr/Subst Abuse Services',
 '564 North Idaho Road Suite 9']

In [903]:
cn_sa = "FortMcDowellIndianCommunityFamilyandCommunityServicesYavapaiRoad"
split_center_street(cn_sa)

['Fort McDowell Indian Community Familyand Community Services', 'Yavapai Road']

In [904]:
cn_sa = "VentressCorrectionalFacilitySubstanceAbuseServicesStateRoad239"
split_center_street(cn_sa)

['Ventress Correctional Facility Substance Abuse Services State', 'Road 239']

In [905]:
def write_data(writer, data, year, pagen):
    '''
    Write data to csv
    '''
    for city in data:
        cityname = city[0]
        lencn = len(cityname.replace(' ', ''))
        for facil in city[1:]:
            print("\n", facil)
            data_holder = [year, pagen]
            keys = facil[1]
            facil = facil[0]
            #Center name
            if "," in facil:
                center_name_city, address = facil.split(",")
            else:
                center_name_city, address = facil.split(".")
            #City
            data_holder += split_center_street(center_name_city[:-lencn])
            #State
            data_holder.append(cityname)
            data_holder.append(address[:2])
            #Postal code, Phone
            if address[7] == "-":
                data_holder.append(address[2:12].replace("O", "0").replace("S", "5"))
                data_holder.append(address[12:].replace("O", "0").replace("S", "5"))
            else:
                data_holder.append(address[2:7].replace("O", "0").replace("S", "5"))
                data_holder.append(address[7:].replace("O", "0").replace("S", "5"))
            #Keys
            data_holder.append(keys)
            writer.writerow(data_holder)
    return data_holder[4]

In [906]:
def run(year, start_page, end_page = 0):

    #read file
    dir_filename = '{0}/directory_{0}.pdf'.format(year)
    pdfFileObj = open(dir_filename, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    
    #pages to scrape
    if end_page == 0:
        end_page = pdfReader.numPages - 1

    save_as_filename = '{0}/{0}.csv'.format(year)
    if os.path.isfile(save_as_filename):
        with open(save_as_filename, 'a', newline='') as file:
            writer = csv.writer(file)
    else:
        with open(save_as_filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Year", "Page", "Center_name", "Street_address", "City", \
                             "State", "Postal_code", "Phone", "Keys"])
    #scrape
    firstcity = "Alexandar City"
    for pagen in range(start_page, end_page + 1):
        #print(pagen)
        rawdata = pdfReader.getPage(pagen - 1).extractText()
        
        data = split_data(rawdata, firstcity)
        
        with open(save_as_filename, 'a', newline='') as file:
            writer = csv.writer(file)
            firstcity = write_data(writer, data, year, pagen)

In [853]:
run(1998, 12, 561) #12


 ['LtghthouseofTallapoosaCountyInc36FranklinStreetAlexanderCity,AL35010(205)234-4894', 'TX RR HH']

 ['AnnistonFellowshipHouseInc106East22ndStreetAnniston,AL36201(256)236-7229', 'TX RR HH']

 ['CJCalhoun/CleburncMentalHealthCenterNewDirections331East8thStreetAnniston,AL36202(256)236-3403Hotlines:(256)236-3403(256)236-8003', 'TX PV OS OR IO AD DD SS CM']

 ['AlcoholismRecoveryServicesInc2701JeffersonAvenueSWBirmingham,AL35211(205)923-6552', 'TX PV OS DT OR IO RD RR HV SS OH MD']

 ['AletheiaHouse201FinleyAvenueWestBirmingham,AL35204(205)324-6502', 'TX OS OR IO SS MD']

 ['BirminghamHealthCareforTheHomeless71225thStreetNorthBirmingham.AL35203(205)439-7216', 'TX PV OS OR IO SS']

 ['FGBradfordHealthServicesBirminghamRegionalOfficeJefferson631BeaconParkwayWestSuite211Birmingham,AL35209(800)293-7191', 'TX OS OR IO AD DD']

 ['SSDepartmentofVeteransAffairsMedicalCenter171711thAvenueSouthBirmingham,AL3520S(205)939-2135', 'TX OS OR DD GH']

 ['FGFellowshipHouseInc162512thAvenueSouthBirmingham

IndexError: string index out of range

In [842]:
year = 1998
pagen = 12
firstcity = "Alexandar City"
rawdata = pdfReader.getPage(pagen - 1).extractText()
data = split_data(rawdata, firstcity)
with open(save_as_filename, 'a', newline='') as file:
    writer = csv.writer(file)
    clean_data(writer, data, year, pagen)

In [913]:
pagen = 14
rawdata = pdfReader.getPage(pagen - 1).extractText()
rawdata

'JacksonHospitalPsychiatricUnit1235ForestAvenueMontgomery,AL36106(334)293-8660TXDT/ID/GH/MCMDFGPILighthouseCounselingCenterIncIntensiveOutpatientUnit1415EastSouthBoulevardMontgomery,AL36116(334)286-5980TXPVOS/ORIO/DDHVPW/SS/MDPIMeadhavenAddictiveDiseaseProgram2105EastSouthBoulevardMontgomery,AL36116(334)286-3535TXPVOSDT/ODIOIDIR/DD/GH/MCFGPIMontgomeryMetroTreatmentCenter4303NormanBridgeRoadMontgomery,AL36105(334)288-5363TXMLPVOSDT/ODORIO/SSGHSGMUSCLESHOALSROGERSVILLETheFreedomHouseRoute4Rogersvillc,AL35652(205)247-1222TX/10RR/PW/SSOHHHRUSSELLVILLESunriseLodgeSubstanceAbuseTreatmentCenter1163WashingtonAvenueSWRussellville,AL3S653(205)332-0078TX/RR/SS/MDPlSELMACahabaCaresSubstanceAbuseServices912JeffDavisAvenueSelma,AL36701(334)874-2600Hotline:(334)875-2109TXPVOS/ORIO/ADHVPW/SSCJ/MDPhoenixHouseInc70035thAvenueTuscaloosa,AL35401(205)758-3867TX/RR/HH/FGTuscaloosaTreatmentCenterS3SRiverRoadNESuiteG-3Tuscaloosa,AL35404(205)507-0800TXMLOSDT/ODOR/SS/PIVeteransAffairsMedicalCenterPrimaryCareSub

In [914]:
data = split_data(rawdata, "Montgomery")

[['', ['JacksonHospitalPsychiatricUnit1235ForestAvenueMontgomery,AL36106(334)293-8660', 'TX DT ID GH MC MD FG PI'], ['LighthouseCounselingCenterIncIntensiveOutpatientUnit1415EastSouthBoulevardMontgomery,AL36116(334)286-5980', 'TX PV OS OR IO DD HV PW SS MD PI'], ['MeadhavenAddictiveDiseaseProgram2105EastSouthBoulevardMontgomery,AL36116(334)286-3535', 'TX PV OS DT OD IO ID IR DD GH MC FG PI'], ['MontgomeryMetroTreatmentCenter4303NormanBridgeRoadMontgomery,AL36105(334)288-5363', 'TX ML PV OS DT OD OR IO SS GH SG'], ['MUSCLESHOALSROGERSVILLETheFreedomHouseRoute4Rogersvillc,AL35652(205)247-1222TX/10', 'RR PW SS OH HH'], ['RUSSELLVILLESunriseLodgeSubstanceAbuseTreatmentCenter1163WashingtonAvenueSWRussellville,AL3S653(205)332-0078', 'TX RR SS MD PI'], ['SELMACahabaCaresSubstanceAbuseServices912JeffDavisAvenueSelma,AL36701(334)874-2600Hotline:(334)875-2109', 'TX PV OS OR IO AD HV PW SS CJ MD'], ['PhoenixHouseInc70035thAvenueTuscaloosa,AL35401(205)758-3867', 'TX RR HH FG'], ['TuscaloosaTreatme

IndexError: string index out of range

In [235]:
data = [['MaxwellAirForceBaseSubstanceAbuseProgram42MDOS/SGOMH330KirkpatrickAvenueEMaxwellAFB,AL36113-6334(334)493-4534TXPVOSDT/ODORIO/PW/SSGHCM/FGMOBILE'],
 ['BradfordHealthServicesMobileOutreachMobile1000HillcrestRoadSuite304Mobile,AL36695(800)333-0906Hotlines:(334)633-0900(800)333-0906TXPVOS/ORIO/SS'],
 ['DauphinWayLodge1009DauphinStreetMobile,AL36604(256)438-4729TXOS/IORR/SSHH/FGECD'],
 ['Program29SOSpringhillAvenueMobile,AL36607(334)476-1677TX/IO/SS'],
 ['Oasis4211GovernmentBoulevardMobile,AL36693(334)666-2569TXOS/OR10/SSOH/PIMONTGOMERY'],
 ['BradfordHealthServicesMontgomeryOutreachMontgomery100MendelParkwayMontgomery,AL36117(800)873-2887Hotline:(334)244-0702TXPVOS/ORIO/AD/SS'],
 ['ChemicalAddictionsProgramInc1153AirBaseBoulevardMontgomery,AL36108(334)265-4544Hotline:(800)762-3790TXOS/IORR/AD/SS/MDFGALABAMA']]

In [236]:
save_as_filename = '{0}/{0}.csv'.format(year)

with open(save_as_filename, 'a', newline='') as file:
    writer = csv.writer(file)
    clean_data(writer, data, year, pagen)


 MaxwellAirForceBaseSubstanceAbuseProgram42MDOS/SGOMH330KirkpatrickAvenueEMaxwellAFB,AL36113-6334(334)493-4534TXPVOSDT/ODORIO/PW/SSGHCM/FGMOBILE

 BradfordHealthServicesMobileOutreachMobile1000HillcrestRoadSuite304Mobile,AL36695(800)333-0906Hotlines:(334)633-0900(800)333-0906TXPVOS/ORIO/SS

 DauphinWayLodge1009DauphinStreetMobile,AL36604(256)438-4729TXOS/IORR/SSHH/FGECD

 Program29SOSpringhillAvenueMobile,AL36607(334)476-1677TX/IO/SS

 Oasis4211GovernmentBoulevardMobile,AL36693(334)666-2569TXOS/OR10/SSOH/PIMONTGOMERY

 BradfordHealthServicesMontgomeryOutreachMontgomery100MendelParkwayMontgomery,AL36117(800)873-2887Hotline:(334)244-0702TXPVOS/ORIO/AD/SS

 ChemicalAddictionsProgramInc1153AirBaseBoulevardMontgomery,AL36108(334)265-4544Hotline:(800)762-3790TXOS/IORR/AD/SS/MDFGALABAMA


- Check each term that has something blanck